## 의미 검색 유사도

### 데이터 준비

In [1]:
# https://huggingface.co/datasets/klue/klue/viewer/mrc
from datasets import load_dataset

klue_mrc_dataset = load_dataset('klue', 'mrc', split='train')

README.md:   0%|          | 0.00/22.5k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/17554 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5841 [00:00<?, ? examples/s]

In [2]:
klue_mrc_dataset

Dataset({
    features: ['title', 'context', 'news_category', 'source', 'guid', 'is_impossible', 'question_type', 'question', 'answers'],
    num_rows: 17554
})

In [3]:
# 일부 데이터만 사용 약 1000행 정도
data_size = 1000
klue_mrc_dataset_train = klue_mrc_dataset.train_test_split(train_size=data_size, 
                                                    shuffle=False)['train']
klue_mrc_dataset_train

Dataset({
    features: ['title', 'context', 'news_category', 'source', 'guid', 'is_impossible', 'question_type', 'question', 'answers'],
    num_rows: 1000
})

In [4]:
klue_mrc_dataset_train[10]
klue_mrc_dataset_train.shape

(1000, 9)

### 데이터 임베딩

In [5]:
from sentence_transformers import SentenceTransformer

model_sentence = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS")
model_sentence

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.02k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/336k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/967k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [6]:
embeddings = model_sentence.encode(klue_mrc_dataset_train['context'])
embeddings

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

array([[ 0.5222372 , -1.1391715 ,  0.12654728, ..., -1.1193498 ,
         0.03521844, -0.6324905 ],
       [-0.36406684, -0.6319616 ,  0.19000682, ..., -0.21136579,
         0.4021174 ,  0.38462663],
       [-0.36406684, -0.6319616 ,  0.19000682, ..., -0.21136579,
         0.4021174 ,  0.38462663],
       ...,
       [-0.92905575, -0.72629637, -0.37646997, ..., -0.05437021,
         0.72452426, -0.45626414],
       [-0.9430958 , -1.0330368 ,  0.25581682, ..., -0.24693918,
         0.28673935, -0.35674646],
       [-0.8397217 , -0.68244684, -0.07826207, ..., -0.31085145,
        -0.05979928, -0.7542277 ]], dtype=float32)

In [7]:
embeddings.shape

(1000, 768)

### 검색 인덱스 생성 (KNN)

In [8]:
!pip install faiss-cpu faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 50.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 17.4 MB/s eta 0:00:00:00:0100:01


In [9]:
import faiss # meta api : vector distance 

index_KNN = faiss.IndexFlatL2(embeddings.shape[1]) # KNN 알고리즘 초기화, vocab 사이즈 만큼 vector 공간 할당
index_KNN

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7e553c2f9bc0> >

In [10]:
# 인덱스에 embedding 저장, 일종의 테이블 생성(메모리용 vectorDB 유사)
# 신규 단어 처리 어떻게? db는 로컬에 저장?
index_KNN.add(embeddings)

In [11]:
type(index_KNN),index_KNN

(faiss.swigfaiss_avx2.IndexFlatL2,
 <faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7e553c2f9bc0> >)

In [12]:
## 활용 
query = '이번 연도에는 언제 비가 많이 올까?'

embedding_query = model_sentence.encode([query])
embedding_query.shape

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

(1, 768)

In [13]:
distances, indices = index_KNN.search(embedding_query, 3)
distances, indices

(array([[300.91895, 420.8628 , 420.8628 ]], dtype=float32),
 array([[  0, 920, 921]]))

In [14]:
for idx in indices[0]:
    print(klue_mrc_dataset_train['context'][idx]) # 학습 데이터가 많아야 성능 좋다. 꼭 의미가 안맞아도 거리 가까운게 높게 나온다 
    print('--------------------------------')

올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2~3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다.장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24~25일이었으며 장마기간은 32일, 강수일수는 17.2일이었다.기상청은 올해 장마기간의 평균 강수량이 350~400㎜로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 지장이 없을 전망이다.
--------------------------------
연구 결과에 따르면, 오리너구리의 눈은 대부분의 포유류보다는 어류인 칠성장어나 먹장어, 그 중에서도 태평양먹장어(Eptatretus stoutii)와 구조가 비슷하다. 게다가 오리너구리의 눈에서는 같은 포유류에게서 찾아보기 힘든 이중 원추세포가 발견된다. 

크게 발달하지도 않았고 물 속에서도 시야 확보를 위하여 쓰이지 않는 눈이지만, 오리너구리의 조상뻘 되는 종들이 한때 시각에 보다 크게 의존했다는 증거는 많이 남아 있다

### 라마 api 벡터 활용 

In [16]:
!pip install llama-index llama-index-embeddings-huggingface -qqq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.6/250.6 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.1/463.1 kB 20.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.9.1 which is incompatible.


In [17]:
from llama_index.core import VectorStoreIndex, Document 

In [18]:
# llama에 맞는 데이터셋 형식 구성
text_list = klue_mrc_dataset[:100]['context']
# len(text_list), text_list[99]
documents = [ Document(text=text) for text in text_list] # llama_index 형식
len(documents), documents[99]

(100,
 Document(id_='f9d9899f-ece8-49ac-8aca-99a034233ad8', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='국내 연구진이 발암 가능성이 없는 맞춤형 줄기세포로 척수 신경을 만드는 데 성공했다. 김정범 울산과학기술원(UNIST) 생명과학부 교수(사진) 연구진은 피부 세포에 ‘Oct4’라는 유전자를 집어넣어 척수 신경 세포로 곧바로 분화시키는 데 성공했다고 27일 발표했다. 신체의 다양한 부위로 분화하는 배아줄기세포나 유도만능줄기세포(iPSc)는 분화 과정에서 암세포로 돌변할 가능성이 크지만, 이 맞춤형 줄기세포는 그럴 가능성이 없어 부작용이 없는 환자 맞춤형 치료에 활용할 수 있다. 야마나카 신야 일본 교토대 교수는 체세포에 4개 특정 유전자를 넣어 성체 세포의 시간을 거꾸로 돌려 배아줄기세포처럼 모든 부위로 분화하는 iPSc를 개발했다. 두 만능세포는 분화 과정에서 암세포가 생길 수 있다. 연구진은 iPSc에서 사용된 유전자 가운데 Oct4를 쥐의 피부 세포에 넣어 척수 신경을 둘러싼 보호막인 신경 수초를 구성하는 척수 세포를 만들었다. 신체 다양한 부위로 분화하지 않고 척수 세포나 혈액 세포처럼 특정 세포로만 콕 찍어 분화하는 줄기세포를 만들었다.', path=None, url=None, mimetype=None), image_resource=None, audio_resource=None, video_resource=None, text_template='{metadata_str}\n\n{content}'))

In [19]:
# llama_index vectorizing 
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embedding_model = HuggingFaceEmbedding(model_name='snunlp/KR-SBERT-V40K-klueNLI-augSTS') 
embedding_model

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.02k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/336k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/967k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

HuggingFaceEmbedding(model_name='snunlp/KR-SBERT-V40K-klueNLI-augSTS', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7e55050a9300>, num_workers=None, max_length=128, normalize=True, query_instruction=None, text_instruction=None, cache_folder=None)

In [20]:
## index 화
index_llama = VectorStoreIndex.from_documents(documents,
                                 embed_model=embedding_model)
index_llama

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [21]:
# 활용
query_engine = index_llama.as_query_engine(llm=None) # default openai

ValueError: 
******
Could not load OpenAI model. If you intended to use OpenAI, please check your OPENAI_API_KEY.
Original error:
No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys

To disable the LLM entirely, set llm=None.
******

In [ ]:
query_text = f'이번 연도에는 언제 비가 많이 올까?'
response = query_engine.query(query_text)

In [ ]:
response